In [23]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
def readraw_color(filename, width, height, num_channels): 
    dtype = np.uint8
    with open(filename, 'rb') as f:
        raw_image_data = np.frombuffer(f.read(), dtype=dtype)
    image_data = raw_image_data.reshape((height, width, num_channels))
    return image_data.tolist()
def grayscale(name):
    for i in range(0, len(name)): 
        for a in range(0, len(name[0])):   
            Y = name[i][a][0]
            # Append the grayscale value to the pixel's color tuple twice.
            name[i][a].append(Y)
            name[i][a].append(Y)
def plot_img(img, title, directory):
    plt.imshow(img)  # Display image
    plt.title(title)  # Set figure title
    plt.axis('off')  # Hide axis

    if directory != False:  # Save file if directory provided
        plt.savefig(directory)
    plt.show() 
background_building = readraw_color('./Project2_Images/building.raw', 256, 256, 1)
grayscale(background_building)
def padding(img, mask_coefficient, y, x, noise, colour, thres, dev):
    mask_size_1 = int(np.sqrt(len(mask_coefficient)))
    half_mask_size_1 = mask_size_1 // 2
    result = [0, 0, 0] if colour else 0
    k = 0

    for i in range(-half_mask_size_1, half_mask_size_1 + 1):
        for j in range(-half_mask_size_1, half_mask_size_1 + 1):
            y_idx = y+i
            x_idx = x+j
            pixel_value = img[y_idx][x_idx] if 0 <= y_idx < len(img) and 0 <= x_idx < len(img[0]) else ([0, 0, 0] if colour else [0])
            result += pixel_value[0] * mask_coef[k]
            k += 1
            
    result = result / dev
        
    return result

def edge_detection(img, name, mask_coefficient, mask_coefficient2, title, dev, noise=True, colour=False, thres=200):
    img_edge_detected_x = copy.deepcopy(img)
    img_edge_detected_y = copy.deepcopy(img)

    # Applying Sobel filter in X direction
    for i in range(len(img_edge_detected_x)):
        for j in range(len(img_edge_detected_x[0])):
            Y = padding(img, mask_coefficient, i, j, noise, colour, thres, dev)
            for c in range(3):
                img_edge_detected_x[i][j][c] = Y
                
    # Applying Sobel filter in Y direction
    for i in range(len(img_edge_detected_y)):
        for j in range(len(img_edge_detected_y[0])):
            Y = padding(img, mask_coefficient2, i, j, noise, colour, thres, dev)
            for c in range(3):
                img_edge_detected_y[i][j][c] = Y
                
    # Calculating gradient magnitude and direction
    gradient_magnitude = copy.deepcopy(img)

    for i in range(len(gradient_magnitude)):
        for j in range(len(gradient_magnitude[0])):
            for c in range(3):
                Gx = img_edge_detected_x[i][j][c]
                Gy = img_edge_detected_y[i][j][c]
                mag = math.sqrt(Gx**2 + Gy**2)
                if mag > thres:
                    gradient_magnitude[i][j][c] = 250
                else:
                    gradient_magnitude[i][j][c] = 0
    plotting_before_after(img, gradient_magnitude, name, title)
    return gradient_magnitude
def plotting_before_after(img, img_noise_removed, name, title):
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].imshow(img)
    axs[0].axis('off')
    axs[0].set_title(f'{name} Before {title} Detection')
    
    axs[1].imshow(img_noise_removed)
    axs[1].axis('off')
    axs[1].set_title(f'{name} After {title} Detection')
    
    # plt.savefig(f'./output_images/Problem 3/{name}/{name} Before and After {title}.png')
    
    plt.show()  

# Mask for Sobel Row Gradient Edge Detection
mask_coef_x = [1, 0, -1,
            2, 0, -2,
            1, 0, -1]

# Mask for Sobel Column Gradient Edge Detection
mask_coef_y = [-1, -2, -1,
            0, 0, 0,
            1, 2, 1]


dev = 4

thres = 17.5

sobel_fist = edge_detection(background_building, "Background",title="Sobel Row Gradient", dev=dev, mask_coef=mask_coef_x, mask_coef2=mask_coef_y, thres=thres)
def bitwise_and(image1, image2):

    # Perform element-wise logical AND
    result = np.logical_and(np.array(image1), np.array(image2)).astype(np.uint8) * 255

    return result
    
def edge_detection_2nd_deriv(img, name, mask_coef, title, dev, sobel_fist, noise=True, colour=False, thres = 200):
    img_edge_detected = copy.deepcopy(img)

    # First loop - applying the mask and getting the gradient
    for i in range(len(img_edge_detected)):
        for j in range(len(img_edge_detected[0])):
            Y = mask_padding(img, mask_coef, i, j, noise, colour, thres, dev)

            for c in range(3):
                img_edge_detected[i][j][c] = Y
                
    # Second loop - zero crossing detection
    for i in range(1, len(img_edge_detected)-1):
        for j in range(1, len(img_edge_detected[0])-1):
            
            for c in range(3):
                current_pixel = img_edge_detected[i][j][c]
                right_pixel = img_edge_detected[i][j+1][c]
                bottom_pixel = img_edge_detected[i+1][j][c]

                # Zero crossing check for grayscale images
                if (current_pixel * right_pixel < 0) or (current_pixel * bottom_pixel < 0):
                    Y = 255
                else:
                    Y = 0

                img_edge_detected[i][j][c] = Y
    
    res = bitwise_and(sobel_fist, img_edge_detected)
    
    plotting_before_after(img, res , name, title)
    return img_edge_detected
second_dev_mask = [0, 0, -1, 0, 0, 
                   0, -1, -2, -1, 0,
                  -1, -2, 16, -2, -1, 
                   0, -1, -2, -1, 0,
                   0, 0, -1, 0, 0]


edge_detection_2nd_deriv(background_building, "Background",title="5x5 LoG", dev=1, mask_coef=second_dev_mask, sobel_fist=sobel_fist)
background_building_noise = readraw_color('images/building_noise.raw', 256, 256, 1)
transform_to_gray(background_building_noise)
plot_img(background_building_noise, "Original Noise Building", False)
def mask_padding_noise(img, mask_coef, y, x, noise, colour):
    mask_size = int(np.sqrt(len(mask_coef)))
    half_mask_size = mask_size // 2
    result = [0, 0, 0] if colour else 0
    k = 0

    for i in range(-half_mask_size, half_mask_size + 1):
        for j in range(-half_mask_size, half_mask_size + 1):
            y_idx = y+i
            x_idx = x+j
            
            # Provide 0 values for padding (outside image boundaries)
            pixel_value = img[y_idx][x_idx] if 0 <= y_idx < len(img) and 0 <= x_idx < len(img[0]) else ([0, 0, 0] if colour else [0])
            
            if colour:
                for c in range(3):
                    result[c] += pixel_value[c] * mask_coef[k]
            else:
                result += pixel_value[0] * mask_coef[k]
                
            k += 1

    if noise:
        if colour:
            for c in range(3):
                result[c] = result[c] / sum(mask_coef)
        else:
            result = result / sum(mask_coef)

    if colour:
        return [int(val) for val in result]
    else:
        return int(result)

def noise_removal_padding(img, name, mask_coef, title='Noise With Padding', noise=True, colour=False):
    img_noise_removed = copy.deepcopy(img)

    for i in range(len(img_noise_removed)):
        for j in range(len(img_noise_removed[0])):
            Y = mask_padding_noise(img, mask_coef, i, j, noise, colour)
            if colour:
                for c in range(3):
                    img_noise_removed[i][j][c] = Y[c]
            else:
                for c in range(3):
                    img_noise_removed[i][j][c] = Y

    plotting_before_after(img, img_noise_removed, name, title)
    return img_noise_removed

# Function to calculate the histogram of an image
def image_histogram(img):
    # Flatten the image array into a 1D array of pixel values
    pixel_values = np.array(img).flatten()
    # Initialize a histogram with 256 entries (for each pixel intensity from 0 to 255)
    histogram = [0] * 256
    # Increment the histogram count for each pixel's intensity
    for pixel in pixel_values:
        histogram[pixel] += 1
    
    return histogram
    
# Function to find the minimum pixel intensity with a non-zero count in the histogram
def find_fmin(histogram):
    # Iterate over histogram to find the first non-zero value
    for idx in range(0, len(histogram)):
        if histogram[idx] != 0:
            return idx  # Return the index (pixel intensity) where the first non-zero value occurs

# Function to find the maximum pixel intensity with a non-zero count in the histogram
def find_fmax(histogram):
    # Iterate over histogram in reverse to find the last non-zero value
    for idx in range(len(histogram)-1, -1, -1):
        if histogram[idx] != 0:
            return idx  # Return the index (pixel intensity) where the last non-zero value occurs
        
# Function to compute the stretched pixel intensity value
def h(F, Fmin, Fmax, Gmin, Gmax):
    # Calculate the slope of the line used for stretching the pixel intensity values
    slope = 255 / (Fmax - Fmin)
    # Calculate how far the pixel's intensity is from the minimum value
    x = (F - Fmin)
    # Gmin is typically 0 in histogram equalization, added here for flexibility
    b = Gmin
    # Return the new stretched pixel value
    return int(slope * x + b)
def linear_contrast(img, name):
    # Create a copy of the original image to avoid modifying it
    img_enhance = copy.deepcopy(img)
    # Calculate the histogram of the original image
    histogram = image_histogram(img_enhance)
    # Find the minimum and maximum pixel intensities with non-zero counts
    Fmin = find_fmin(histogram)
    print("Fmin =", Fmin)
    Fmax = find_fmax(histogram)
    print("Fmax =", Fmax)
    # Set the desired minimum and maximum pixel intensities after enhancement
    Gmax = 255
    Gmin = 0
    
    # Initialize an empty list to store mapping of original to enhanced pixel values
    linear = []

    # Iterate over each pixel in the image
    for i in range(0, len(img_enhance)): 
        for a in range(0, len(img_enhance[0])):
            # Apply the linear transformation to the pixel's intensity
            Y = h(img_enhance[i][a][0], Fmin, Fmax, Gmin, Gmax)
            
            # Store the original and transformed pixel values
            linear.append([Y, img_enhance[i][a][0]])
            
            # Clip the transformed intensity if it's outside the desired range
            if Y > Gmax:
                Y = Gmax
            elif Y < Gmin:
                Y = Gmin
            
            # Set the enhanced pixel intensity for all color channels (convert to grayscale)
            img_enhance[i][a][0] = Y
            img_enhance[i][a][1] = Y
            img_enhance[i][a][2] = Y
    
    # Calculate the histogram of the enhanced image
    histogram_after = image_histogram(img_enhance)
    
    # Plot the original and enhanced images side by side
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[1].imshow(img_enhance)
    axs[1].axis('off')
    axs[1].set_title(f'{name}  after contrast')
    axs[0].imshow(img)
    axs[0].axis('off')
    axs[0].set_title(f'{name}  before contrast')
    # plt.savefig(f'./output_images/Problem 2/Full Range Linear Scaling Method/{name} Rose/Original and Enhanced Rose.png')
     
    plt.show()
    
    return img_enhance
# Mask for Noise Removal
mask_coef = [1, 2,1,
            2, 4, 2,
            1, 2, 1]

img_noise_removed = noise_removal_padding(background_building_noise, "Building Noise Removal", mask_coef)
img_enhenced = linear_contrast(img_noise_removed, "Building Noise")
dev = 4
thres = 45
sobel_noise = edge_detection(img_enhenced, "Background",title="Sobel Row Gradient", dev=dev, mask_coef=mask_coef_x, mask_coef2=mask_coef_y, thres=thres)
edge_detection_2nd_deriv(img_enhenced, "Background",title="5x5 LoG", dev=1, mask_coef=second_dev_mask, sobel_fist=sobel_noise)
second_dev_mask = [0, 0, -1, 0, 0, 
                   0, -1, -2, -1, 0,
                  -1, -2, 16, -2, -1, 
                   0, -1, -2, -1, 0,
                   0, 0, -1, 0, 0]



TypeError: edge_detection() got an unexpected keyword argument 'mask_coef'